In [ ]:
# @title Setup
from google.colab import auth
# from google.cloud import bigquery
from google.colab import data_table
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datetime import datetime, timedelta

# Verfications

In [ ]:
verifications_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/verifications/verifications.parquet')

In [ ]:
verifications_df = verifications_df.sort_values(by='fid')
verifications_df.head()

,fid,address,timestamp,deleted_at
29197,1,0x86924c37a93734e8611eb081238928a9d18a63c0,2023-07-22 00:08:54+00:00,NaT
30132,2,0x91031dcfdea024b4d51e775486111d2b2a715871,2023-04-28 17:42:20+00:00,NaT
30131,2,0xf86a7a5b7c703b1fd8d93c500ac4cc75b67477f0,2024-02-22 22:24:00+00:00,NaT
228,2,anjftzzfdupt8amg1shgtujvhx6uefgr34vram1qcuqw,2024-02-22 23:52:06+00:00,2024-02-22 23:52:06+00:00
30130,2,9t92xzy9q5syfkbh4rzwedfxjazkgzj5pgviphktbjyy,2024-02-22 23:54:35+00:00,NaT


# Links Data


In [ ]:
links_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/links/links.parquet')

In [ ]:
links_df = links_df[links_df.type.isin(['follow', 'unfollow'])]
links_df = links_df.drop_duplicates(['target_fid', 'fid'], keep=False)

In [ ]:
links_df.head()

,fid,timestamp,target_fid,type,deleted_at
0,19099,2024-04-20 22:06:08+00:00,461009,follow,NaT
1,252634,2024-05-03 10:06:31+00:00,250833,follow,NaT
2,418937,2024-04-06 09:26:16+00:00,415186,follow,NaT
3,385938,2024-04-17 19:43:30+00:00,403156,follow,NaT
4,500406,2024-04-29 11:21:25+00:00,484567,follow,NaT


# Profile Data

In [ ]:
profiles_df = pd.read_parquet(f'/content/drive/My Drive/farcaster_bigquery/profiles/profiles.parquet')

In [ ]:
profiles_df.head()

,fid,last_updated_at,data,custody_address,date
0,504692,2024-04-28 08:29:33+00:00,None,0x77b0352100ebdb86a45557296e05721e48cc6d01,2024-04-28
1,530839,2024-05-11 10:39:33+00:00,None,0xef7c5ee8eb10cd4ae60611506532e1a6a279c070,2024-05-11
2,521202,2024-05-08 11:05:41+00:00,None,0x74d8967e812de34702ecd3d453a44bf37440b10b,2024-05-08
3,530583,2024-05-11 08:40:01+00:00,None,0xfefa4f4aebb13ebe75ac5ffb7374ea6c2478e350,2024-05-11
4,531932,2024-05-12 02:14:13+00:00,None,0x3fe713ae9fbcb4f86f960716e6316d418d15e6a2,2024-05-12


# Convert tabular data to graph data

In [ ]:
G = nx.from_pandas_edgelist(
    links_df,
    source='fid',
    target='target_fid',
    create_using=nx.DiGraph()
)

# Run Pagerank on the fids




In [ ]:
pr = nx.pagerank(G)

# Process results to find top 10 fids by pagerank

In [ ]:
df_pr = pd.DataFrame.from_dict(pr,  orient='index', columns = ['pagerank']).reset_index(names=['fid'])

In [ ]:
df_pr

,fid,pagerank
0,19099,1.225519e-05
1,461009,2.612976e-06
2,252634,6.516458e-06
3,250833,1.405970e-06
4,418937,1.592415e-06
...,...,...
410099,189247,4.503864e-07
410100,185724,4.503864e-07
410101,164852,4.503864e-07
410102,248489,4.503864e-07


In [ ]:
pr_sorted_df = df_pr.sort_values(by='pagerank', ascending=False)

In [ ]:
pr_sorted_profiles_df = pd.merge(profiles_df, pr_sorted_df, on = 'fid').sort_values('pagerank', ascending=False)

In [ ]:
# pr_sorted_profiles_df.to_parquet('/content/drive/My Drive/farcaster_bigquery/pagerank_profiles.parquet')
# pr_sorted_profiles_df.to_csv('/content/drive/My Drive/farcaster_bigquery/pagerank_profiles.csv')
pr_sorted_profiles_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/pagerank_profiles.parquet')

# Map the pagerank to a 50-99 scale

In [ ]:
# Scaling from 50-99
min_target = 50
max_target = 99

original_min = pr_sorted_profiles_df['pagerank'].min()
original_max = pr_sorted_profiles_df['pagerank'].max()

pr_sorted_profiles_df['pr_scaled_50_99'] = round((pr_sorted_profiles_df['pagerank'] - original_min) * (max_target - min_target) / (original_max - original_min) + min_target, 0)

In [ ]:
# Scarling from 1-99
min_target = 1
max_target = 99

original_min = pr_sorted_profiles_df['pagerank'].min()
original_max = pr_sorted_profiles_df['pagerank'].max()

pr_sorted_profiles_df['pr_scaled_1_99'] = round((pr_sorted_profiles_df['pagerank'] - original_min) * (max_target - min_target) / (original_max - original_min) + min_target,0)

In [ ]:
pr_sorted_profiles_df['pr_scaled_1_99_lt50is50'] = pr_sorted_profiles_df['pr_scaled_1_99'].apply(lambda x: max(x, 50))

In [ ]:
pr_sorted_profiles_df.head()

,fid,last_updated_at,data,custody_address,date,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50
381284,3,2023-08-31 16:52:50+00:00,"{""bio"":""Working on Farcaster and Warpcast."",""d...",None,2023-08-31,0.006092,99.0,99.0,99.0
392789,5650,2024-01-19 01:20:58+00:00,"{""bio"":""hullo"",""display"":""Vitalik Buterin"",""pf...",None,2024-01-19,0.005517,90.0,94.0,90.0
263907,99,2024-04-16 01:55:58+00:00,"{""bio"":""@base contributor #001; onchain cities...",None,2024-04-16,0.004769,78.0,88.0,78.0
371557,2,2024-05-06 21:10:43+00:00,"{""bio"":""Technowatermelon. Elder Millenial. Bui...",None,2024-05-06,0.004576,75.0,87.0,75.0
260989,207,2024-04-12 20:29:26+00:00,"{""bio"":""(summer of) protocol {support | guild ...",None,2024-04-12,0.004384,72.0,85.0,72.0


In [ ]:
pr_sorted_profiles_df.data.fillna('{}', inplace=True)

In [ ]:
# Function to parse JSON strings
def parse_json_column(json_str):
    return pd.Series(json.loads(json_str))

# Apply the function to the 'Profile' column
expanded_columns = pr_sorted_profiles_df['data'].apply(parse_json_column)

# Concatenate the original DataFrame with the new columns
scores_df = pd.concat([pr_sorted_profiles_df, expanded_columns], axis=1)

# Drop the original 'Profile' column
scores_df.drop(columns=['data', 'url'], inplace=True)

In [ ]:
scores_df['percentile'] = round(scores_df['pagerank'].rank(pct=True) * 99)
scores_df['percentile_50_99'] = scores_df['percentile'].apply(lambda x: max(x, 50))

In [ ]:
scores_df['percentile_50_99']

381284    99.0
392789    99.0
263907    99.0
371557    99.0
260989    99.0
          ... 
179411    50.0
179394    50.0
327306    50.0
179310    50.0
249113    50.0
Name: percentile_50_99, Length: 407054, dtype: float64

In [ ]:
scores_df.to_csv('/content/drive/My Drive/farcaster_bigquery/pagerank_percentile.csv')
scores_df.sample(100).to_csv('/content/drive/My Drive/farcaster_bigquery/pagerank_percentile_random100.csv')
scores_df[0:100].to_csv('/content/drive/My Drive/farcaster_bigquery/pagerank_percentile_top100.csv')

In [ ]:
# scores_df[0:100].to_csv('/content/drive/My Drive/farcaster_bigquery/pagerank_top100.csv')

In [ ]:
scores_df.head()

,fid,last_updated_at,custody_address,date,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50,bio,display,pfp,username,percentile,percentile_50_99
381284,3,2023-08-31 16:52:50+00:00,None,2023-08-31,0.006092,99.0,99.0,99.0,Working on Farcaster and Warpcast.,Dan Romero,https://res.cloudinary.com/merkle-manufactory/...,dwr.eth,99.0,99.0
392789,5650,2024-01-19 01:20:58+00:00,None,2024-01-19,0.005517,90.0,94.0,90.0,hullo,Vitalik Buterin,https://i.imgur.com/IzJxuId.jpg,vitalik.eth,99.0,99.0
263907,99,2024-04-16 01:55:58+00:00,None,2024-04-16,0.004769,78.0,88.0,78.0,@base contributor #001; onchain cities w/ OAK ...,Jesse Pollak 🔵,https://i.imgur.com/rOy7TtZ.gif,jessepollak,99.0,99.0
371557,2,2024-05-06 21:10:43+00:00,None,2024-05-06,0.004576,75.0,87.0,75.0,Technowatermelon. Elder Millenial. Building Fa...,Varun Srinivasan,https://i.seadn.io/gae/sYAr036bd0bRpj7OX6B-F-M...,v,99.0,99.0
260989,207,2024-04-12 20:29:26+00:00,None,2024-04-12,0.004384,72.0,85.0,72.0,(summer of) protocol {support | guild | fellow...,timbeiko.eth,https://lh3.googleusercontent.com/G2z3yvOBkfHn...,tim,99.0,99.0


In [ ]:
scores_df.size

4884648

In [ ]:
scores_df.describe()

,fid,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50,percentile,percentile_50_99
count,407054.0,4.070540e+05,407054.000000,407054.000000,407054.000000,407054.000000,407054.000000
mean,326442.712999,2.452689e-06,1.022093,50.009814,50.000531,49.492544,62.126325
std,134890.81742,3.813109e-05,0.613998,0.306662,0.124169,28.573282,15.784895
min,1.0,4.469975e-07,1.000000,50.000000,50.000000,0.000000,50.000000
25%,232044.25,4.529686e-07,1.000000,50.000000,50.000000,25.000000,50.000000
50%,335054.5,4.768670e-07,1.000000,50.000000,50.000000,50.000000,50.000000
75%,437403.75,8.084265e-07,1.000000,50.000000,50.000000,74.000000,74.000000
max,539339.0,6.091768e-03,99.000000,99.000000,99.000000,99.000000,99.000000


In [ ]:
scores_df[0:100]

,fid,last_updated_at,custody_address,date,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50,bio,display,pfp,username,percentile,percentile_50_99
381284,3,2023-08-31 16:52:50+00:00,None,2023-08-31,0.006092,99.0,99.0,99.0,Working on Farcaster and Warpcast.,Dan Romero,https://res.cloudinary.com/merkle-manufactory/...,dwr.eth,99.0,99.0
392789,5650,2024-01-19 01:20:58+00:00,None,2024-01-19,0.005517,90.0,94.0,90.0,hullo,Vitalik Buterin,https://i.imgur.com/IzJxuId.jpg,vitalik.eth,99.0,99.0
263907,99,2024-04-16 01:55:58+00:00,None,2024-04-16,0.004769,78.0,88.0,78.0,@base contributor #001; onchain cities w/ OAK ...,Jesse Pollak 🔵,https://i.imgur.com/rOy7TtZ.gif,jessepollak,99.0,99.0
371557,2,2024-05-06 21:10:43+00:00,None,2024-05-06,0.004576,75.0,87.0,75.0,Technowatermelon. Elder Millenial. Building Fa...,Varun Srinivasan,https://i.seadn.io/gae/sYAr036bd0bRpj7OX6B-F-M...,v,99.0,99.0
260989,207,2024-04-12 20:29:26+00:00,None,2024-04-12,0.004384,72.0,85.0,72.0,(summer of) protocol {support | guild | fellow...,timbeiko.eth,https://lh3.googleusercontent.com/G2z3yvOBkfHn...,tim,99.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381286,302,2023-12-22 04:45:24+00:00,None,2023-12-22,0.001154,20.0,59.0,50.0,Working on Farcaster.,Goksu Toprak,https://i.imgur.com/YM5spql.jpg,gt,99.0,99.0
372249,4407,2023-12-05 22:52:08+00:00,None,2023-12-05,0.001150,19.0,59.0,50.0,"The Farcaster formerly known as Kchamp. Books,...",Katherine,https://i.imgur.com/kynnpYw.jpg,keccers.eth,99.0,99.0
404851,9816,2024-05-09 10:46:16+00:00,None,2024-05-09,0.001122,19.0,59.0,50.0,🇫🇷 Generative artist • he/him • \nNFTs on gm. ...,Camille Roux,https://i.imgur.com/KIWqYJC.jpg,camilleroux,99.0,99.0
400878,1689,2024-02-13 22:13:17+00:00,None,2024-02-13,0.001118,19.0,59.0,50.0,"wannabe cypherpunk, engineer working on @frame...",Stephan,https://i.imgur.com/njEtQOM.jpg,stephancill,99.0,99.0


In [ ]:
scores_df.sample(n=100)

,fid,last_updated_at,custody_address,date,pagerank,pr_scaled_1_99,pr_scaled_50_99,pr_scaled_1_99_lt50is50,bio,display,pfp,username,percentile,percentile_50_99
216491,186205,NaT,None,None,4.503864e-07,1.0,50.0,50.0,NaN,NaN,NaN,NaN,12.0,50.0
301205,200400,2023-12-08 20:51:33+00:00,None,2023-12-08,4.545731e-07,1.0,50.0,50.0,Fit via vi,JT,https://i.imgur.com/cNKn7bT.jpg,jtchristy,28.0,50.0
128326,512768,2024-05-09 07:48:38+00:00,0xb4556f353b5cba3eb388193c629a5a4700d5254e,2024-05-09,1.249790e-06,1.0,50.0,50.0,Ma name Sheyda,Shyda,https://imagedelivery.net/BXluQx4ige9GuW0Ia56B...,NaN,81.0,81.0
165444,389985,2024-04-16 17:59:37+00:00,0xb9f3a09ea02c699298db900ed544b5b8182fd732,2024-04-16,9.428426e-07,1.0,50.0,50.0,crypto✨,Rey,https://imagedelivery.net/BXluQx4ige9GuW0Ia56B...,winter3,77.0,77.0
27503,414802,2024-03-28 17:39:22+00:00,0x2985b820f13373cf5baf5b400af2fb6ae4d07ff6,2024-03-28,4.594605e-07,1.0,50.0,50.0,,Octopus,https://i.imgur.com/IUlfS5w.jpg,octopusonbush,36.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213007,181689,NaT,None,None,4.503864e-07,1.0,50.0,50.0,NaN,NaN,NaN,NaN,12.0,50.0
202373,176729,NaT,None,None,4.503864e-07,1.0,50.0,50.0,NaN,NaN,NaN,NaN,12.0,50.0
119479,435185,2024-04-01 13:53:03+00:00,0xde840ce806499a84effc6ea91a28c24ffbac5ca3,2024-04-01,5.359724e-07,1.0,50.0,50.0,"I’m pepe artist, check me on X \n@BillyNFTees",BillyNFTees,https://i.imgur.com/yVrNhrS.jpg,christianbicasso,62.0,62.0
391048,274187,2024-02-11 01:26:20+00:00,None,2024-02-11,1.051224e-06,1.0,50.0,50.0,"have a joint family in which my uncle, grandpa...",Manish Sutariya,https://dego.finance/upload/big/1.png,manishr,79.0,79.0
